# Regressão Linear com NumPy

Nessa tarefa você vai estender sua implementação da tarefa passada para considerar múltiplas variáveis. Você pode estender a versão vetorizada implementada neste notebook para regressão simples. 
- Rode o algoritmo nesses dados, onde as linhas representam as notas de alunos de computação de alunos da UFCG em algumas disciplinas do primeiro período. A última coluna é a variável alvo representando o CRA final depois de concluir o curso. As outras colunas são algumas disciplinas do primeiro período. O pressuposto aqui é que as notas em disciplinas no primeiro período ajudam a explicar o CRA final dos alunos de computação.__
- Compare o valor dos coeficientes estimados pelo seu algoritmo com o valor dos coeficientes da regressão linear do scikit learn para testar se o seu algoritmo está funcionando corretamente.

A entrega deve ser o link no seu github para o notebook Jupyter com código python e texto explicativo quando necessário. De preferência, crie um repositório na sua conta do github e envie o link do html do notebook.

In [ ]:
import numpy as np
import math
import time

### Versão Não Vetorizada
Função para calcular o MSE (Mean Squared Error):

$MSE(\hat{w}) = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i (x_i))^2$

In [ ]:
# y = mx + b
# m is slope, b is y-intercept
def compute_mse(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        totalError += (y - (m * x + b)) ** 2
    return totalError / float(len(points))

Função para fazer uma atualização dos parâmetros no Gradiente Descendente:

$w_0 = w_0 + 2\alpha\sum_{i=1}^N (y_i - (w_0+w_1x_i))$

$w_1 = w_1 + 2\alpha\sum_{i=1}^N x_i(y_i - (w_0+w_1x_i))$

In [ ]:
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        b_gradient += (y - ((m_current * x) + b_current))
        m_gradient += x * (y - ((m_current * x) + b_current))
    new_b = b_current + (2 * learningRate * b_gradient)
    new_m = m_current + (2 * learningRate * m_gradient)
    return [new_b, new_m, b_gradient, m_gradient]

$||\mathbf{w}||_2 = \sqrt{\sum_{j=1}^D w_j^2}$

In [ ]:
def norm_2(x):
    c=0
    for i in range(len(x)):
        c += x[i]**2
    return math.sqrt(c)

Função para iterar sobre o gradiente descendente até convergência.

In [ ]:
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, epsilon):
    b = starting_b
    m = starting_m
    grad = np.array([np.inf,np.inf])
    i = 0
    while (norm_2(grad)>=epsilon):
        b, m, b_gradient, m_gradient = step_gradient(b, m, points, learning_rate)
        grad = np.array([b_gradient,m_gradient])
        if i % 1000 == 0:
            #print(norm_2(grad))
            print("MSE na iteração {0} é de {1}".format(i,compute_mse(b,m,points)))
        i+= 1
    return [b, m]

In [ ]:
points = np.genfromtxt("income.csv", delimiter=",")
learning_rate = 0.0001
initial_b = 0 # initial y-intercept guess
initial_m = 0 # initial slope guess
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_mse(initial_b, initial_m, points)))
print("Running...")
tic = time.time()
[b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, erro = {2}".format(b, m, compute_mse(b, m, points)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + "ms")

### Versão Vetorizada

$MSE(\hat{w})=\frac{1}{N}(y-\hat{\mathbf{w}}^T\mathbf{x})^T(y-\hat{\mathbf{w}}^T\mathbf{x})$

In [71]:
import numpy as np
import math
import time

In [91]:
def compute_mse_vectorized(w, X, Y):
    #print("w {0}, X {1}, Y {2}".format(w, X, Y))
    res = Y - np.dot(X, w).sum(axis=1) # lidando com mais de um X: SOMA? todos os valores de X (linha) para cada Y
    totalError = np.dot(res.T, res)
    return totalError / float(len(Y))

In [109]:
def step_gradient_vectorized(w_current, X, Y, learningRate):
    res = Y - np.dot(X, w_current)
    b_gradient = np.sum(res)
    X = X[:, :6]
    
    m_gradients = [b_gradient]
    new_w = [(w_current[0] + (2 * learningRate * b_gradient))]
    for var in range(1, len(w_current)): # para cada variável, deve-se calcular o gradiente e o novo parâmetro w
        var_gradient = np.sum(np.multiply(res, X[:, var])) # o gradiente da variável é o somatório do erro multiplicado pelos valores da variável
        print("var {0}".format(var_gradient))
        m_gradients.append(var_gradient) # TODO O CÁLCULO DO GRADIENTE PARECE ESTAR INCORRETO, DESPENCANDO, VAMOS FREÁ-LO
        
        var_w = w_current[var] + (2 * learningRate * var_gradient)
        new_w.append(var_w)
    
    return [new_w, m_gradients]

In [100]:
def gradient_descent_runner_vectorized(starting_w, X, Y, learning_rate, epsilon):
    w = starting_w
    grad = np.array([np.inf, np.inf])
    i = 0
    while (np.linalg.norm(grad) >= epsilon):
        w, m_gradients = step_gradient_vectorized(w, X, Y, learning_rate)
        #grad = np.array([b_gradient,m_gradient])
        grad = m_gradients
        #print(np.linalg.norm(grad))
        print("MSE na iteração {0} é de {1}".format(i, compute_mse_vectorized(w, X, Y)))
        if i % 1000 == 0:
            print("MSE na iteração {0} é de {1}".format(i, compute_mse_vectorized(w, X, Y)))
        i+= 1
    return w

In [101]:
def run():
    points = np.genfromtxt("sample_treino.csv", delimiter=",")
    points = np.c_[np.ones(len(points)),points]
    X = points[1:, 1:-1] # independent variables
    X = np.c_[np.ones(len(X)), X] # convert array to matrix
    Y = points[1:, 5] # dependent variable
    init_w = np.zeros((6,1)) # alterando para 6 linhas e 1 coluna
    learning_rate = 0.0001
    #num_iterations = 10000
    epsilon = 0.5
    
    print("Executando...")
    tic = time.time()
    w = gradient_descent_runner_vectorized(init_w, X, Y, learning_rate, epsilon)
    toc = time.time()
    
    print("Gradiente descendente convergiu com erro = {0} e:".format(compute_mse_vectorized(w,X,Y)))
    for var in range(0, len(w)):
        print("w_{0} = {1}".format(var, w[var]))
    
    print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")


In [110]:
if __name__ == '__main__':
    run()

Executando...
var 355461.04000000004
var 416282.24
var 366518.24
var 402118.64
var 322505.04
MSE na iteração 0 é de 8075762.763557907
MSE na iteração 0 é de 8075762.763557907
var -157764651.71641856
var -185816568.36681402
var -162320510.65190226
var -179080181.06709635
var -138556665.9652824
MSE na iteração 1 é de 1581877912243.1995
var 69826087281.73602
var 82240713585.26147
var 71842845328.07544
var 79259660839.35431
var 61329267830.50928
MSE na iteração 2 é de 3.098806958691215e+17
var -30904978104695.293
var -36399684204758.9
var -31797593475299.805
var -35080271865154.773
var -27144287103437.195
MSE na iteração 3 é de 6.070381372166603e+22
var 1.3678521814288024e+16
var 1.6110474911475586e+16
var 1.407359275684164e+16
var 1.5526503928085672e+16
var 1.2014042597313464e+16
MSE na iteração 4 é de 1.1891521647440645e+28
var -6.054104241592396e+18
var -7.130484990985364e+18
var -6.228962365994448e+18
var -6.872019839888339e+18
var -5.317406897800129e+18
MSE na iteração 5 é de 2.329479